In [1]:
! unzip Teeth_DataSet.zip

Streaming output truncated to the last 5000 lines.
  inflating: Teeth_Dataset/Validation/CoS/b_111_0_6094.jpg  
  inflating: Teeth_Dataset/Testing/CoS/b_150_0_2905.jpg  
  inflating: Teeth_Dataset/Testing/OT/ot_1241_0_1157.jpg  
  inflating: Teeth_Dataset/Validation/CoS/b_95 - Copy_0_6372.jpg  
  inflating: Teeth_Dataset/Testing/CaS/a_152_0_7749.jpg  
  inflating: Teeth_Dataset/Testing/CaS/a_126_0_9339.jpg  
  inflating: Teeth_Dataset/Testing/outputs/a_110_0_9820.jpg  
  inflating: Teeth_Dataset/Testing/outputs/a_141_0_7620.jpg  
  inflating: Teeth_Dataset/Validation/CoS/b_132_0_2266.jpg  
  inflating: Teeth_Dataset/Testing/outputs/a_99_0_9803.jpg  
  inflating: Teeth_Dataset/Testing/OT/ot_1223_0_6812.jpg  
  inflating: Teeth_Dataset/Testing/OT/ot_1223_0_6082.jpg  
  inflating: Teeth_Dataset/Testing/output/a_145_0_219.jpg  
  inflating: Teeth_Dataset/Testing/output/a_119_0_1965.jpg  
  inflating: Teeth_Dataset/Testing/CoS/b_112_0_1990.jpg  
  inflating: Teeth_Dataset/Testing/output/a_1

In [2]:
! pip install torch torchvision matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import os

# Data augmentation for training
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor()
])

# No augmentation for validation/testing
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Paths
data_dir = "./Teeth_Dataset"
train_dir = os.path.join(data_dir, "Training")
val_dir = os.path.join(data_dir, "Validation")
test_dir = os.path.join(data_dir, "Testing")

# Datasets
train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(val_dir, transform=test_transform)
test_dataset = datasets.ImageFolder(test_dir, transform=test_transform)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

class_names = train_dataset.classes
num_classes = len(class_names)
print("Classes:", class_names)


Classes: ['CaS', 'CoS', 'Gum', 'MC', 'OC', 'OLP', 'OT']


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # -> [16, 112, 112]
        x = self.pool(F.relu(self.conv2(x)))  # -> [32, 56, 56]
        x = x.view(-1, 32 * 56 * 56)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate model
model = SimpleCNN(num_classes=num_classes)
print(model)


SimpleCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=100352, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=7, bias=True)
)


In [5]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [6]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct, total = 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_acc = 100 * correct / total
        val_acc = evaluate(model, val_loader)

        print(f"Epoch {epoch+1}: Loss={running_loss:.4f}, Train Acc={train_acc:.2f}%, Val Acc={val_acc:.2f}%")

def evaluate(model, data_loader):
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    return 100 * correct / total


In [7]:
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

Epoch 1: Loss=196.6600, Train Acc=20.99%, Val Acc=27.14%
Epoch 2: Loss=171.7072, Train Acc=29.71%, Val Acc=33.17%
Epoch 3: Loss=163.6608, Train Acc=34.89%, Val Acc=39.01%
Epoch 4: Loss=151.1410, Train Acc=40.75%, Val Acc=43.29%
Epoch 5: Loss=138.1899, Train Acc=46.61%, Val Acc=45.23%
Epoch 6: Loss=129.5285, Train Acc=50.28%, Val Acc=50.00%
Epoch 7: Loss=116.9347, Train Acc=55.91%, Val Acc=53.70%
Epoch 8: Loss=108.5221, Train Acc=59.18%, Val Acc=55.74%
Epoch 9: Loss=99.9552, Train Acc=62.26%, Val Acc=59.82%
Epoch 10: Loss=90.1668, Train Acc=67.22%, Val Acc=61.38%


In [8]:
# Evaluate on test set
test_accuracy = evaluate(model, test_loader)
print(f"Test Accuracy: {test_accuracy:.2f}%")


Test Accuracy: 42.04%
